In [1]:
import os
import sys
import torch
import accimage
import numpy as np
import pandas as pd
from PIL import Image
from imageio import imread
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms, set_image_backend, get_image_backend
import torch.nn.functional as F
import pickle
import train_utils
import data_utils
import os
import gc
import sys
import torch
import psutil
import pickle
import numpy as np
import pandas as pd
import torch.nn as nn
from sklearn import metrics
from collections import Counter
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import models, set_image_backend
set_image_backend('accimage')
import data_utils
import train_utils
root_dir_coad = '/n/mounted-data-drive/COAD/'

%reload_ext autoreload
%autoreload 2

In [2]:
sa_train, sa_val = data_utils.load_COAD_train_val_sa_pickle()
root_dir = '/n/mounted-data-drive/COAD/'
magnification = '10.0'
batch_type = 'slide'

train_transform = train_utils.transform_train
val_transform = train_utils.transform_validation


In [15]:
train_set = data_utils.TCGADataset_tiled_slides(sa_train, root_dir, transform=train_transform, magnification=magnification)
train_loader = DataLoader(train_set, batch_size=256, pin_memory=True, num_workers=32)


val_set = data_utils.TCGADataset_tiled_slides(sa_val, root_dir, transform=val_transform, magnification=magnification)
val_loader = DataLoader(val_set, batch_size=256, pin_memory=True, num_workers=32)

In [4]:
state_dict_file = '/n/tcga_models/resnet18_WGD_10x.pt'
device = torch.device('cuda', 0)
output_shape = 1

In [5]:
resnet = models.resnet18(pretrained=False)
resnet.fc = nn.Linear(2048, output_shape, bias=True)
saved_state = torch.load(state_dict_file, map_location=lambda storage, loc: storage)
resnet.load_state_dict(saved_state)
resnet.fc = nn.Linear(2048, 2048, bias=False)
resnet.fc.weight.data = torch.eye(2048)
resnet.cuda(device=device)
for param in resnet.parameters():
    param.requires_grad = False

In [6]:
def pool_fn(x):
    #v,a = torch.max(x,0)
    v = torch.mean(x,0)
    return v


slide_level_classification_layer = nn.Linear(2048,1)
slide_level_classification_layer.cuda()

Linear(in_features=2048, out_features=1, bias=True)

In [7]:
e = 0
learning_rate = 1e-4
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(slide_level_classification_layer.parameters(), lr = learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, min_lr=1e-8)

In [11]:
train_utils.tcga_tiled_slides_training_loop(e, train_loader, resnet, 
                                    slide_level_classification_layer, criterion, 
                                 optimizer, pool_fn)

Epoch: 0, Step: 0, Slide Number: 20, Train Batch NLL: 0.6677
Epoch: 0, Step: 1, Slide Number: 30, Train Batch NLL: 0.7208
Epoch: 0, Step: 2, Slide Number: 35, Train Batch NLL: 0.6314
Epoch: 0, Step: 3, Slide Number: 46, Train Batch NLL: 0.7131
Epoch: 0, Step: 4, Slide Number: 53, Train Batch NLL: 0.6682
Epoch: 0, Step: 5, Slide Number: 60, Train Batch NLL: 0.6581
Epoch: 0, Step: 6, Slide Number: 69, Train Batch NLL: 0.6737
Epoch: 0, Step: 7, Slide Number: 81, Train Batch NLL: 0.6465
Epoch: 0, Step: 8, Slide Number: 100, Train Batch NLL: 0.6728
Epoch: 0, Step: 9, Slide Number: 114, Train Batch NLL: 0.7716
Epoch: 0, Step: 10, Slide Number: 133, Train Batch NLL: 0.7215
Epoch: 0, Step: 11, Slide Number: 154, Train Batch NLL: 0.7501
Epoch: 0, Step: 12, Slide Number: 164, Train Batch NLL: 0.6630
Epoch: 0, Step: 13, Slide Number: 169, Train Batch NLL: 0.5819
Epoch: 0, Step: 14, Slide Number: 176, Train Batch NLL: 0.6041
Epoch: 0, Step: 15, Slide Number: 188, Train Batch NLL: 0.5952
Epoch: 0, 

In [16]:
train_utils.tcga_tiled_slides_validation_loop(e, val_loader, resnet, 
                                    slide_level_classification_layer, criterion, 
                                 scheduler, pool_fn)

Epoch: 0, Step: 0, Slide Number: 81, Val Total NLL: 0.6701, Val Accuracy: 0.61
